### Imports

In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Activation, Embedding, LSTM
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import plotly as py
import plotly.graph_objs as go
from plotly import tools
from matplotlib.pyplot import figure

Using TensorFlow backend.


### Data Load

In [71]:
data = pandas.read_csv('./data/Learning/brake, acc/Train_Set.csv')
test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')

x_train_set = data[['x','y','z']]

target = data['AccStatus']

x_test_set = test[['x','y','z']]
y_test_set = test['AccStatus']


x_train_set = np.array(x_train_set,dtype=float)          
target = np.array(target,dtype=float)

x_test_set =  np.array(x_test_set,dtype=float)
y_test_set =  np.array(y_test_set,dtype=float)

In [5]:
def SmoothData(data):
    size = data.shape[0] 
    dim = data.shape[1]
    
    smooth_data = np.zeros((size,dim))
    
    smooth_data[0] = data[0]
    smooth_data[1] = data[1]
    smooth_data[size-1] = data[size-1]
    smooth_data[size-2] = data[size-2]
    
    for i in range(2, size-2):
        smooth_data[i] = np.average([data[i], data[i-1], data[i+1]], axis=0)
    return smooth_data

In [73]:
x_train_set = SmoothData(x_train_set)
x_test_set = SmoothData(x_test_set)

In [50]:
train_byx = data[['x']]

target_byx = data['AccStatus']

test_byx = test[['x']]
y_test_byx = test['AccStatus']


train_byx = np.array(train_byx, dtype=float)          
target_byx = np.array(target_byx, dtype=float)

test_byx =  np.array(test_byx, dtype=float)
y_test_byx =  np.array(y_test_byx, dtype=float)


### One-hot encoding

In [74]:
target_onehot  = []
y_test_set_onehot = []
for i in range(len(target)):
    if(target[i]==0):
        target_onehot.append(np.array([1,0,0]))
    elif (target[i]==1):
        target_onehot.append(np.array([0,1,0]))
    elif(target[i]==2):
        target_onehot.append(np.array([0,0,1]))
        
for i in range(len(y_test_set)):
    if(y_test_set[i]==0):
        y_test_set_onehot.append(np.array([1,0,0]))
    elif (y_test_set[i]==1):
        y_test_set_onehot.append(np.array([0,1,0]))
    elif (y_test_set[i]==2):
        y_test_set_onehot.append(np.array([0,0,1]))

### LSTM one-hot

In [78]:
model = Sequential()
model.add(LSTM(100, input_shape = (50,3), return_sequences = True, dropout=0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='RMSprop',metrics=['categorical_accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 50, 100)           41600     
_________________________________________________________________
dense_7 (Dense)              (None, 50, 3)             303       
Total params: 41,903
Trainable params: 41,903
Non-trainable params: 0
_________________________________________________________________


In [79]:
history = model.fit( 
    np.array([x_train_set[i:i+50] for i in range(0,46950,20)]),
    np.array([target_onehot[i:i+50] for i in range(0,46950,20)]),
    epochs=200,
    validation_data= (np.array([x_train_set[i:i+50] for i in range(47000,47600,20)]),
                     np.array([target_onehot[i:i+50] for i in range(47000,47600,20)])),
batch_size=64)


Train on 2348 samples, validate on 30 samples
Epoch 1/200
2348/2348 [==============================] - 5s 2ms/step - loss: 0.4949 - categorical_accuracy: 0.8940 - val_loss: 0.1490 - val_categorical_accuracy: 1.0000
Epoch 2/200
2348/2348 [==============================] - 2s 901us/step - loss: 0.3656 - categorical_accuracy: 0.9174 - val_loss: 0.1114 - val_categorical_accuracy: 1.0000
Epoch 3/200
2348/2348 [==============================] - 2s 913us/step - loss: 0.3380 - categorical_accuracy: 0.9173 - val_loss: 0.1149 - val_categorical_accuracy: 1.0000
Epoch 4/200
2348/2348 [==============================] - 2s 891us/step - loss: 0.3258 - categorical_accuracy: 0.9175 - val_loss: 0.0783 - val_categorical_accuracy: 1.0000
Epoch 5/200
2348/2348 [==============================] - 3s 1ms/step - loss: 0.3046 - categorical_accuracy: 0.9203 - val_loss: 0.0692 - val_categorical_accuracy: 1.0000
Epoch 6/200
2348/2348 [==============================] - 2s 913us/step - loss: 0.2986 - categorical_acc

Epoch 96/200
2348/2348 [==============================] - 2s 1ms/step - loss: 0.2016 - categorical_accuracy: 0.9394 - val_loss: 0.0153 - val_categorical_accuracy: 1.0000
Epoch 97/200
2348/2348 [==============================] - 2s 897us/step - loss: 0.2119 - categorical_accuracy: 0.9380 - val_loss: 0.0211 - val_categorical_accuracy: 1.0000
Epoch 98/200
2348/2348 [==============================] - 2s 893us/step - loss: 0.2028 - categorical_accuracy: 0.9399 - val_loss: 0.0136 - val_categorical_accuracy: 1.0000
Epoch 99/200
2348/2348 [==============================] - 2s 897us/step - loss: 0.2018 - categorical_accuracy: 0.9407 - val_loss: 0.0150 - val_categorical_accuracy: 1.0000
Epoch 100/200
2348/2348 [==============================] - 2s 904us/step - loss: 0.1972 - categorical_accuracy: 0.9403 - val_loss: 0.0121 - val_categorical_accuracy: 1.0000
Epoch 101/200
2348/2348 [==============================] - 2s 901us/step - loss: 0.2013 - categorical_accuracy: 0.9389 - val_loss: 0.0209 - v

2348/2348 [==============================] - 2s 817us/step - loss: 0.1724 - categorical_accuracy: 0.9469 - val_loss: 0.0080 - val_categorical_accuracy: 1.0000
Epoch 191/200
2348/2348 [==============================] - 2s 1ms/step - loss: 0.1715 - categorical_accuracy: 0.9485 - val_loss: 0.0115 - val_categorical_accuracy: 1.0000
Epoch 192/200
2348/2348 [==============================] - 2s 839us/step - loss: 0.1756 - categorical_accuracy: 0.9453 - val_loss: 0.0101 - val_categorical_accuracy: 1.0000
Epoch 193/200
2348/2348 [==============================] - 2s 823us/step - loss: 0.1866 - categorical_accuracy: 0.9441 - val_loss: 0.0092 - val_categorical_accuracy: 1.0000
Epoch 194/200
2348/2348 [==============================] - 2s 836us/step - loss: 0.1744 - categorical_accuracy: 0.9465 - val_loss: 0.0079 - val_categorical_accuracy: 1.0000
Epoch 195/200
2348/2348 [==============================] - 2s 834us/step - loss: 0.1901 - categorical_accuracy: 0.9421 - val_loss: 0.0091 - val_categor

In [80]:
x_test_set.shape

(14178, 3)

In [84]:
results = model.predict(np.array(x_test_set[:14150]).reshape(283,50,3))
results = results.reshape(14150,3)

In [85]:
result_class = []
for i in results:
    if(np.argmax(i)==0):
        result_class.append(0)
    elif(np.argmax(i)==1):
        result_class.append(1)
    elif(np.argmax(i)==2):
        result_class.append(2)

In [30]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test_set[:14150], result_class, labels=[0,1], average=None)

(array([0.97409135, 0.87155963]),
 array([0.97901942, 0.62706271]),
 array([0.97654917, 0.7293666 ]),
 array([13441,   303], dtype=int64))

In [86]:

fig =tools.make_subplots(rows=1,cols=1)

result =  go.Scatter(
    x = list(range(0,15000)),
    y = result_class,
    mode = 'lines',
    name='model_result'
)

test =  go.Scatter(
    x = list(range(0,15000)),
    y = y_test_set,
    mode = 'lines',
    name='test'
)

fig.append_trace(result,1,1)
fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Brake,Acc Model Classification')


py.offline.plot(fig, filename='brake,acc_onehot_50_sigmoid_categorical_crossentropy')


'brake,acc_onehot_50_sigmoid_categorical_crossentropy.html'

In [ ]:

test = pandas.read_csv('./data/Learning/brake, acc/Test_Set.csv')


fig =tools.make_subplots(rows=1,cols=1)


test =  go.Scatter(
    x = list(range(0,15000)),
    y = test['x'],
    mode = 'lines',
    name='test'
)

fig.append_trace(test,1,1)


fig['layout'].update(height=500, title = 'Test Data')


py.offline.plot(fig, filename='brake_acc_test_data')


### Sensor Log Generated Data

In [11]:
my_data = pandas.read_csv('./MyData/my_iOS_device_2019-04-05_13-32-24_+0430.csv')

working_set = my_data[['accelerometerAccelerationX(G)','accelerometerAccelerationY(G)','accelerometerAccelerationZ(G)']][3200:3700]        

set_user = my_data[['motionUserAccelerationX(G)','motionUserAccelerationY(G)','motionUserAccelerationZ(G)']]    





In [12]:
set_user = np.array(working_set_user)
set_user = SmoothData(working_set_user)



In [28]:
working_set_user = set_user[3200:3700]
len(set_user)

11917

In [29]:
fig =tools.make_subplots(rows=1,cols=1)


acx =  go.Scatter(
    x = list(range(0,len(set_user))),
    y = [set_user[i][0] for i in range(0,11917) ],
    mode = 'lines',
    name = 'x'
)

acy =  go.Scatter(
    x = list(range(0,len(set_user))),
    y = [set_user[i][1] for i in range(0,11917) ],
    mode = 'lines',
    name='y'
)

acz =  go.Scatter(
    x = list(range(0,len(set_user))),
    y = [set_user[i][2] for i in range(0,11917) ],
    mode = 'lines',
    name='z'
)


fig.append_trace(acx,1,1)
fig.append_trace(acy,1,1)
fig.append_trace(acz,1,1)


fig['layout'].update(height=500, title = 'Test Data')


py.offline.plot(fig, filename='brake_acc_test_data')

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\tools.py:465: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead

C:\Users\Nima Tavassoli\Anaconda3\envs\tensorflow_env\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `brake_acc_test_data` didn't end with .html. Adding .html to the end of your file.



'brake_acc_test_data.html'